In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import momepy as mm

In [37]:
region_id = 69333

In [38]:

etcs = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/tessellations_chars_{region_id}.parquet')
buildings = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/buildings_chars_{region_id}.parquet')
streets = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/streets_chars_{region_id}.parquet')
nodes = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/nodes_chars_{region_id}.parquet')

In [40]:
### carry out the analysis on connected buildings...

# bgraph = read_parquet(f'/data/uscuni-ulce/processed_data/neigh_graphs/building_graph_{region_id}.parquet')
# component_labels = bgraph.component_labels

# dissolved_tess_cells = etcs.loc[component_labels.index, :].dissolve(component_labels.values, as_index=True)
# dissolved_tess_cells = dissolved_tess_cells.geometry
# empty_tess_cells = etcs[etcs.index < 0].geometry
# etcs = pd.concat((empty_tess_cells, dissolved_tess_cells)).to_frame()


# buildings = buildings.dissolve(component_labels)

In [41]:
etcs['barea'] = buildings.area
etcs['barea'] = etcs['barea'].fillna(0)

In [42]:
street_graph = mm.gdf_to_nx(streets, preserve_index=True)
nodes, edges = mm.nx_to_gdf(street_graph, spatial_weights=False)


In [43]:
blg_nid = mm.get_nearest_street(
    buildings, edges
)

In [44]:
# %%time
# links = mm.get_network_ratio(etcs, edges)
# etcs[['edgeID_keys', 'edgeID_values']] = links
# keys = etcs.edgeID_values.apply(lambda a: np.argmax(a))
# tess_nid = np.array([inds[i] for inds, i in zip(etcs.edgeID_keys, keys)])

# etc_nid = pd.Series(tess_nid, index=etcs.index)

In [45]:
etc_nid = mm.get_nearest_street(
    etcs, edges
)

In [46]:
%%time
inpidxs, tree_idxs = streets.sindex.query(etcs.geometry, predicate='dwithin', distance=1)

CPU times: user 3.37 s, sys: 0 ns, total: 3.37 s
Wall time: 3.37 s


In [47]:
street_etcarea_std = etcs.iloc[inpidxs].area.groupby(streets.index[tree_idxs]).std()
street_etcarea_std = street_etcarea_std.fillna(0)
etc_street_std = etc_nid.map(lambda x: street_etcarea_std.loc[x] if x in street_etcarea_std else 0)

In [48]:
# graph_dir = '/data/uscuni-ulce/processed_data/neigh_graphs/'
# from libpysal.graph import read_parquet, Graph

# graph = read_parquet(graph_dir + f'tessellation_graph_{region_id}.parquet')

In [ ]:
graph = Graph.build_fuzzy_contiguity(etcs.geometry, tolerance=.1) 

In [ ]:
graph3 = graph.higher_order(k=3, lower_order=True, diagonal=True)

In [ ]:
## variety stuff

In [ ]:
# good
etcs['barea'] = buildings.area
etcs['barea'] = etcs['barea'].fillna(0)
assert (etcs.loc[buildings.index, 'barea'] == buildings.area).all()

res = graph.describe(etcs['barea'])
etcs['barea_std'] = res['std']

In [ ]:
res = graph3.describe(etcs['barea'])
etcs['barea3_std'] = res['std']

In [ ]:
res = graph.describe(etcs['sdcAre'])
etcs['tarea_std'] = res['std']

In [ ]:
enc_etcarea_std = etcs.area.groupby(etcs['enclosure_index']).std().fillna(0)
etcs['enc_etcarea_std'] = etcs['enclosure_index'].map(lambda x: enc_etcarea_std.loc[x])

In [ ]:
# not really sensible
street_barea_std = buildings.area.groupby(blg_nid).std()
street_barea_std = street_barea_std.fillna(0)
blg_street_std = blg_nid.map(lambda x: street_barea_std.loc[x])

etcs['blg_street_std'] = blg_street_std
etcs['blg_street_std'] = etcs['blg_street_std'].fillna(0)

In [ ]:
# not really sensible
etcs['etc_street_std'] = etc_street_std.fillna(0)

In [ ]:
## doesnt really add much. mean is basically area
etcs['cor_std'] = res['std']
etcs['cor_mean'] = res['mean']

etcs['cor_std'] = etcs['cor_std'].fillna(0)
etcs['cor_mean'] = etcs['cor_mean'].fillna(0)

res = graph.describe(etcs['cor_std'])
etcs['cor_std'] = res['mean']

res = graph.describe(etcs['cor_mean'])
etcs['cor_mean'] = res['std']

In [ ]:
## nothing much
etcs['sicar'] = etcs['barea'] / etcs.area
res = graph.describe(etcs['sicar'])
etcs['sicar_std'] = res['mean']

In [ ]:
etcs['bperi'] = buildings.length
etcs['bperi'] = etcs['bperi'].fillna(0)
res = graph.describe(etcs['bperi'])
etcs['bperi_std'] = res['std']

In [ ]:
import momepy as mm

In [ ]:
etcs['fr'] = mm.facade_ratio(etcs)

In [ ]:
res = graph.describe(etcs['fr'])
etcs['fr_std'] = res['std']

In [ ]:
enc_etcarea_std = etcs.area.groupby(etcs['enclosure_index']).std().fillna(0)
etcs['enc_etcarea_std'] = etcs['enclosure_index'].map(lambda x: enc_etcarea_std.loc[x])

In [ ]:
from core.cluster_validation import generate_detailed_clusters
morphotopes = generate_detailed_clusters(etcs, path = '/data/uscuni-ulce/val_morphotopes/prague_morphotopes.pq')
morphotope_etcs = etcs.loc[morphotopes.index]
morphotope_etcs['mlabel'] = morphotopes.values

types = pd.Series([0,0,0, 1,1, 2,2,2,2, 3,3], index=[8,134,14, 18,31, 7,5,37,36, 57,58])
types = types.replace({0: 'houses', 1: 'grid', 2: 'apartment blocks', 3: 'industry'})
morphotope_etcs = morphotope_etcs[morphotope_etcs['mlabel'].isin(types.index)]
morphotope_etcs['types'] = morphotope_etcs['mlabel'].map(lambda x: types.loc[x])



In [ ]:
res = morphotope_etcs.iloc[:, 2:-1].groupby(morphotope_etcs['types']).mean()
res.style.background_gradient(axis=0, cmap="BuGn")

In [ ]:
# barea_stds and blg_street_stds, might be useful

In [27]:
### fixed minimum for buildings and tess area to limit all the weird variability... 

In [28]:
import lonboard
from sidecar import Sidecar

In [29]:
plotting = etcs[etcs.index >= 0]

In [30]:
plotting.loc[:, 'barea'] = plotting.barea.clip(1, ).fillna(0)
plotting.loc[:, 'barea_std'] = plotting.barea_std.clip(1, )

In [31]:
layer = lonboard.SolidPolygonLayer.from_geopandas(plotting, opacity=.08)

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


In [32]:
sc = Sidecar(title='tess')
m = lonboard.Map(layer)
with sc:
    display(m)

In [33]:
etcs.columns

Index(['enclosure_index', 'geometry', 'stcOri', 'sdcLAL', 'sdcAre', 'sscCCo',
       'sscERI', 'mtcWNe', 'mdcAre', 'ltcWRB', 'sicCAR', 'stcSAl', 'nID',
       'nodeID', 'barea', 'barea_std', 'barea3_std', 'tarea_std',
       'enc_etcarea_std', 'blg_street_std', 'etc_street_std', 'cor_std',
       'cor_mean', 'sicar', 'sicar_std', 'bperi', 'bperi_std', 'fr', 'fr_std'],
      dtype='object')

In [34]:
column = 'barea3_std'

In [40]:
# from lonboard.colormap import apply_continuous_cmap
# import matplotlib as mpl
# from mapclassify import classify

# classifier = classify(plotting[column], 'quantiles', k=100)
# normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
# vals = normalizer(classifier.yb)
# layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

In [46]:
from core.cluster_validation import get_color
a = np.log(plotting[column])
# quintiles = np.nanpercentile(a, np.arange(1, 100, 1).tolist() + [99.5, 99.9])    
quintiles = np.nanpercentile(a, np.arange(1, 100, 2).tolist() + [99.5, 99.9])    
vals = np.searchsorted(quintiles, a)
layer.get_fill_color = get_color(vals)

In [63]:
locs = [445777, 445835, 445963]

In [64]:
etcs.loc[locs].explore()

In [66]:
blg_nid.loc[locs]

445777    74418.0
445835    74417.0
445963    47646.0
dtype: float64